In [ ]:
import os
import json
from ast import literal_eval
import time
import csv
import pandas as pd
import argparse
import vertexai
from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
)

# Initialize Google Application Credentials and Vertex AI environment
def init_vertex_ai(project_id, location, credentials_path):
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credentials_path
    vertexai.init(project=project_id, location=location)

# Create a function to load a document and format the prompt
def load_and_format_prompt(file_path, question):
    with open(file_path,'r', encoding='utf-8') as file:
        document_content = file.read()
    prompt_template = f''' Given the financial information like the income statement, balance sheet, and cash flow statement and trends of stock price of the company: {document_content}. 
            answer the following questions: {question}
			Answer:
            '''
    return prompt_template

# Function to initialize the model with system instructions
def create_model(model_id):
    model = GenerativeModel(
        model_id,
        system_instruction=[
            "You are good at understanding content in the financial documents and extracting the income statement, balance sheet, and cash flow statement from the document.",  
			"You ar good at capturing the trends of stock price of the company from the given histrical stock price.", 
                    ],
    )
    return model

# Function to generate response
def generate_response(model, prompt, temperature=0.9, top_p=1.0, top_k=32, max_output_tokens=8192):
    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        candidate_count=1,
        max_output_tokens=max_output_tokens,
    )
    
    contents = [prompt]
    response = model.generate_content(
        contents,
        generation_config=generation_config
    )
    return response


def save_response_to_file(response, output_path):
	print(response)
	response_json = json.loads(response)
	with open(output_path, 'w', encoding='utf-8') as output_file:
		# output_file.write(response)
		json.dump(response_json, output_file, indent=4)


def process_document(file_path, model_id, project_id, location, credentials_path, output_folder):

	init_vertex_ai(project_id, location, credentials_path)

	model = create_model(model_id)

	prompt = load_and_format_prompt(file_path)

	response = generate_response(model, prompt)
	# print(response.text)

	# print((response.text[8:-4]))

	output_file_name = os.path.splitext(os.path.basename(file_path))[0] + "_response.json"
	output_file_path = os.path.join(output_folder, output_file_name)

	save_response_to_file(response.text[8:-4], output_file_path)
	print(f"Response saved to {output_file_path}")
     

def process_folder(folder_path, model_id, project_id, location, credentials_path, output_folder, output_csv):
	if not os.path.exists(output_folder):
		os.makedirs(output_folder)
	
	start_time = time.time()
	print('start time:', start_time )

	for file_name in os.listdir(folder_path):
		if file_name.endswith('.txt'):  # Only process .txt files
			file_path = os.path.join(folder_path, file_name)
			print(f"Processing file: {file_path}")
			process_document(file_path, model_id, project_id, location, credentials_path, output_folder)
		continue
	print("Finished processing all files.")
	print(time.time()-start_time)

# def json_to_csv(output_folder, output_csv):
	all_keys = set()

	data_rows = []

	for file_name in os.listdir(output_folder):
		if file_name.endswith('.json'):
			file_path = os.path.join(output_folder, file_name)

			with open(file_path, 'r', encoding='utf-8') as json_file:
				data = json.load(json_file)

				# Add the keys to the header list
				all_keys.update(data.keys())
				
				# Add the row (values) to the data list
				data_rows.append(data)

	with open(output_csv, 'w', newline='', encoding='utf-8') as csv_file:
		writer = csv.DictWriter(csv_file, fieldnames=sorted(all_keys))
		writer.writeheader()

		for row in data_rows:
			writer.writerow(row)

	print(f"CSV file '{output_csv}' created successfully.")

## Driver Code


parser = argparse.ArgumentParser()
    
parser.add_argument('folder_path', type=str, help='Path to the input CSV file containing the DataFrame.')
parser.add_argument('output_folder', type=str, help='The name of the column to be updated.')
parser.add_argument('output_csv', type=str, help='Path to save the updated CSV file.')
args = parser.parse_args()

# folder_path = "text_files"
# output_folder = "output_folder_post_occur_correction2"
# output_csv = "preds_3.csv"
process_folder(
    folder_path=args.folder_path,
    model_id="gemini-1.5-flash-001", 
    project_id="msi-genai-frontdoor-499476", 
    location="us-east1", 
    credentials_path="C:\\Users\\CQTF47\\Downloads\\Dipjyoti RAG POC\\devtest-sa.json",
    output_folder=args.output_folder,
	output_csv=args.output_csv
)
# json_to_csv(output_folder, output_csv)




In [10]:
import requests
import json

url = "https://genai-service.stage.commandcentral.com/app-gateway/api/v2/chat"
headers = {
    "x-msi-genai-api-key":  "m,(w(7-CzS-kbVY8u3AF.JYeAG-7PLyuyP6.x@zj", #"LSgoaIy~a3gL9aJR7C.gU!5q~LNOR9WmVpNj99xy", 
    "Content-Type": "application/json",
    # "X-msi-genai-client": "<some-unique-app-identifier>"
}
data = {
    "userId": "cqtf47@motorolasolutions.com",
    "model": "VertexGemini",
    "prompt": "Who plays the best piano?",
    # "sessionId": "c2e07ae5-4d6b-48e6-b035-6a8aefb57321",
    "datastoreId":  "362a5d9c-65f8-43df-a751-2057708501ca" #"0f45024e-3c7a-4b18-a2fa-c0af896f5ff7",
    # "modelConfig": {
    #     "temperature": 0.5,
    #     "max_tokens": 800,
    #     "top_p": 2,
    #     "frequency_penalty": 0,
    #     "presence_penalty": 0
    # }
}

response = requests.post(url, headers=headers, json=data)
# print(response.json())

In [16]:
import requests
import json

url = "https://genai-service.stage.commandcentral.com/app-gateway/api/v2/chat"
headers = {
    "x-msi-genai-api-key":  "m,(w(7-CzS-kbVY8u3AF.JYeAG-7PLyuyP6.x@zj", #"LSgoaIy~a3gL9aJR7C.gU!5q~LNOR9WmVpNj99xy", 
    "Content-Type": "application/json",
    # "X-msi-genai-client": "<some-unique-app-identifier>"
}
data = {
    "userId": "cqtf47@motorolasolutions.com",
    "model": "VertexGemini",
    "prompt": "Looking at the financial documents of adidas in the datastore. Explain the company's financial health and future prospects?",
    # "sessionId": "c2e07ae5-4d6b-48e6-b035-6a8aefb57321",
    "datastoreId":  "362a5d9c-65f8-43df-a751-2057708501ca" #"0f45024e-3c7a-4b18-a2fa-c0af896f5ff7",
    # "modelConfig": {
    #     "temperature": 0.5,
    #     "max_tokens": 800,
    #     "top_p": 2,
    #     "frequency_penalty": 0,
    #     "presence_penalty": 0
    # }
}

response = requests.post(url, headers=headers, json=data)
print(response)

<Response [200]>


In [18]:
response.json()

{'status': True,
 'success': True,
 'sessionId': 'ba813ffb-bde4-45ac-a364-d16c05ea6441',
 'sessionTitle': 'Looking at the financial documents',
 'msg': "The financial health of adidas in 2023 was significantly impacted by the war in Ukraine, which affected the company's supply chain and retail operations. However, adidas still managed to record a modest revenue increase, reaching €21,457 million in 2023, compared to €18,488 million in 2022.\n\nDespite the challenging environment, adidas generated a positive gross margin but its gross profit margin decreased 2.7 percentage points to 47.8% in 2023, primarily due to the higher percentage of sales in the lower-margin  direct-to-consumer (DTC) channel and increased sales allowances.\n\nThe company is focused on optimizing inventory levels and cost management, which are key areas of focus for improving future profitability. However, adidas' net income from continuing operations was a loss of €254 million in 2023, a significant decline from €

In [19]:
import requests
import json

url = "https://genai-service.stage.commandcentral.com/app-gateway/api/v2/chat"
headers = {
    "x-msi-genai-api-key":  "m,(w(7-CzS-kbVY8u3AF.JYeAG-7PLyuyP6.x@zj", #"LSgoaIy~a3gL9aJR7C.gU!5q~LNOR9WmVpNj99xy", 
    "Content-Type": "application/json",
    # "X-msi-genai-client": "<some-unique-app-identifier>"
}
data = {
    "userId": "cqtf47@motorolasolutions.com",
    "model": "VertexGemini",
    "prompt": "Looking at the financial documents of adidas in the datastore. Project the net revenue and gross profit and EBITDA in 2024.",
    # "sessionId": "c2e07ae5-4d6b-48e6-b035-6a8aefb57321",
    "datastoreId":  "362a5d9c-65f8-43df-a751-2057708501ca" #"0f45024e-3c7a-4b18-a2fa-c0af896f5ff7",
    # "modelConfig": {
    #     "temperature": 0.5,
    #     "max_tokens": 800,
    #     "top_p": 2,
    #     "frequency_penalty": 0,
    #     "presence_penalty": 0
    # }
}

response = requests.post(url, headers=headers, json=data)
print(response.json())

{'status': True, 'success': True, 'sessionId': '5e7ae76b-7f1b-4cdd-b915-65bed91a5346', 'sessionTitle': 'Looking at the financial documents', 'msg': "I can't answer that question. While I have access to the financial statements for adidas in 2020, 2021, and 2022, I don't have any data to project those figures for 2024.  I'm limited to the data I've been trained on. \n", 'data': {'userId': 'cqtf47@motorolasolutions.com', 'model': 'VertexGemini', 'datastoreId': '362a5d9c-65f8-43df-a751-2057708501ca'}, 'valid_response': True, 'initialPrompt': True, 'args': {'model': 'gemini-1.5-flash-001', 'response_type': 'text', 'transaction_id': 'd988160c-f088-4a87-9909-ce051edf6c0e'}}


In [1]:
import requests
import json

url = "https://genai-service.stage.commandcentral.com/app-gateway/api/v2/chat"
headers = {
    "x-msi-genai-api-key":  "m,(w(7-CzS-kbVY8u3AF.JYeAG-7PLyuyP6.x@zj", #"LSgoaIy~a3gL9aJR7C.gU!5q~LNOR9WmVpNj99xy", 
    "Content-Type": "application/json",
    # "X-msi-genai-client": "<some-unique-app-identifier>"
}
data = {
    "userId": "cqtf47@motorolasolutions.com",
    "model": "VertexGemini",
    "prompt": "Looking at the financial documents of adidas in the datastore. Try to Project the net revenue and gross profit and EBITDA in 2023.",
    # "sessionId": "c2e07ae5-4d6b-48e6-b035-6a8aefb57321",
    "datastoreId":  "362a5d9c-65f8-43df-a751-2057708501ca" #"0f45024e-3c7a-4b18-a2fa-c0af896f5ff7",
    # "modelConfig": {
    #     "temperature": 0.5,
    #     "max_tokens": 800,
    #     "top_p": 2,
    #     "frequency_penalty": 0,
    #     "presence_penalty": 0
    # }
}

response = requests.post(url, headers=headers, json=data)
print(response.json())

{'status': True, 'success': True, 'sessionId': '9526739a-4dc4-4712-ae43-617dcc6bf737', 'sessionTitle': 'Looking at the financial documents', 'msg': "The financial documents for adidas in the datastore show net revenue, gross profit, and EBITDA for 2020, 2021, and 2022.  To project these figures for 2023, we can use trend analysis, regression, or other forecasting methods, taking into consideration the current economic climate and industry trends. Here's how we can do it:\n\n**1. Trend Analysis:**\n\n* **Net Revenue:** The net revenue of adidas has shown a consistent growth trend over the past few years. It increased from €18,463 million in 2019 to €19,844 million in 2020, €21,234 million in 2021, and €22,511 million in 2022. This suggests a positive trend.\n* **Gross Profit:** Similarly, gross profit has been steadily increasing over the same period, from €10,644 million in 2020 to €11,766 million in 2021, and €12,457 million in 2022.\n* **EBITDA:** EBITDA has also shown growth, increa

In [2]:
import requests
import json

url = "https://genai-service.stage.commandcentral.com/app-gateway/api/v2/chat"
headers = {
    "x-msi-genai-api-key":  "m,(w(7-CzS-kbVY8u3AF.JYeAG-7PLyuyP6.x@zj", #"LSgoaIy~a3gL9aJR7C.gU!5q~LNOR9WmVpNj99xy", 
    "Content-Type": "application/json",
    # "X-msi-genai-client": "<some-unique-app-identifier>"
}
data = {
    "userId": "cqtf47@motorolasolutions.com",
    "model": "VertexGemini",
    "prompt": "Looking at the financial documents and the stock price data of adidas in the datastore. Try to Project the stocks days high and low on 14/October/2024.",
    # "sessionId": "c2e07ae5-4d6b-48e6-b035-6a8aefb57321",
    "datastoreId":  "362a5d9c-65f8-43df-a751-2057708501ca" #"0f45024e-3c7a-4b18-a2fa-c0af896f5ff7",
    # "modelConfig": {
    #     "temperature": 0.5,
    #     "max_tokens": 800,
    #     "top_p": 2,
    #     "frequency_penalty": 0,
    #     "presence_penalty": 0
    # }
}

response = requests.post(url, headers=headers, json=data)
print(response.json())

{'status': True, 'success': True, 'sessionId': 'a02593dd-3b37-4831-b5d1-ad33f5867e90', 'sessionTitle': 'Looking at the financial documents', 'msg': 'Unfortunately, I can\'t give financial advice or project future stock prices.  That said, let\'s look at the data we have to see what we can learn:\n\n* **Stock Data Trends:** The historical data in "ADSGn Historical Data.csv" shows a lot of volatility in the price of adidas stock, with both significant gains and losses. There\'s no clear, consistent upward or downward trend, which is typical for stocks.\n* **Factors Affecting Prices:** Stock prices are influenced by a wide range of things:\n    * **Company Performance:** Adidas\'s financial performance is a major driver. This data is in the Annual Report, where you\'ll find information on revenues, profits, and other key metrics.\n    * **Economic Conditions:** The overall health of the economy, interest rates, and consumer spending affect demand for athletic goods, influencing the stock.

In [4]:
import requests
import json

url = "https://genai-service.stage.commandcentral.com/app-gateway/api/v2/chat"
headers = {
    "x-msi-genai-api-key":  "m,(w(7-CzS-kbVY8u3AF.JYeAG-7PLyuyP6.x@zj", #"LSgoaIy~a3gL9aJR7C.gU!5q~LNOR9WmVpNj99xy", 
    "Content-Type": "application/json",
    # "X-msi-genai-client": "<some-unique-app-identifier>"
}
data = {
    "userId": "cqtf47@motorolasolutions.com",
    "model": "VertexGemini",
    "prompt": "Let's carefully analyze the stock price data for Adidas stored in the datastore. First, we will review the historical patterns in the stock prices over the past months, paying close attention to the trends in daily highs and lows. Next, we will focus on any recurring patterns or seasonality in the data, such as whether the stock tends to rise or fall on certain days or months. We'll also consider external factors like market conditions or company-specific news that might influence the stock price around this time. Finally, based on these insights, we will apply time series analysis to predict the stock's high and low for 14th October 2024. What are the predicted high and low values for the stock on this date?",
    # "sessionId": "c2e07ae5-4d6b-48e6-b035-6a8aefb57321",
    "datastoreId":  "362a5d9c-65f8-43df-a751-2057708501ca" #"0f45024e-3c7a-4b18-a2fa-c0af896f5ff7",
    # "modelConfig": {
    #     "temperature": 0.5,
    #     "max_tokens": 800,
    #     "top_p": 2,
    #     "frequency_penalty": 0,
    #     "presence_penalty": 0
    # }
}

response = requests.post(url, headers=headers, json=data)
print(response.json())

{'status': True, 'success': True, 'sessionId': '9a1ec24c-750d-4c9e-8e26-64ac940678ad', 'sessionTitle': "Let's carefully analyze the stock", 'msg': "It's not possible to predict future stock prices with certainty, even with historical data and time series analysis.  \n\nHere's why:\n\n* **Market Volatility:** Stock prices are influenced by a multitude of factors, including investor sentiment, economic conditions, global events, and company news. These factors are inherently unpredictable and can shift quickly.\n* **Limited Data:** The provided dataset only covers a short time period (approximately one year). This is not enough to establish robust trends and seasonal patterns, especially considering that the data may be affected by unusual market conditions in 2024.\n* **External Factors:** Time series analysis often assumes that historical patterns will continue. However, major events or company news (like a new product launch or significant financial announcement) can completely disrup

In [5]:
import requests
import json

url = "https://genai-service.stage.commandcentral.com/app-gateway/api/v2/chat"
headers = {
    "x-msi-genai-api-key":  "m,(w(7-CzS-kbVY8u3AF.JYeAG-7PLyuyP6.x@zj", #"LSgoaIy~a3gL9aJR7C.gU!5q~LNOR9WmVpNj99xy", 
    "Content-Type": "application/json",
    # "X-msi-genai-client": "<some-unique-app-identifier>"
}
data = {
    "userId": "cqtf47@motorolasolutions.com",
    "model": "VertexGemini",
    "prompt": "Given the historical stock price data of Adidas from the datastore, analyze the trends in daily highs and lows over the past months. Use time series analysis techniques such as moving averages, seasonal trends, and volatility patterns to forecast the stock's high and low values for 14th October 2024. Don't consider any external factors like market events or economic conditions. Based on this analysis, predict the approximately expected high and low prices for Adidas stock on 14th October 2024.",
    # "sessionId": "c2e07ae5-4d6b-48e6-b035-6a8aefb57321",
    "datastoreId":  "362a5d9c-65f8-43df-a751-2057708501ca" #"0f45024e-3c7a-4b18-a2fa-c0af896f5ff7",
    # "modelConfig": {
    #     "temperature": 0.5,
    #     "max_tokens": 800,
    #     "top_p": 2,
    #     "frequency_penalty": 0,
    #     "presence_penalty": 0
    # }
}

response = requests.post(url, headers=headers, json=data)
print(response.json())

{'status': True, 'success': True, 'sessionId': 'ff6bae3c-5a24-4500-a45e-5774771efddc', 'sessionTitle': 'Given the historical stock price', 'msg': 'Unfortunately, I can\'t directly access or analyze files from your computer, including the "ADSGn Historical Data.csv" file. This means I can\'t perform time series analysis on the Adidas stock data you provided to forecast the high and low prices for October 14th, 2024.\n\nHowever, I can explain how time series analysis techniques work and how they would be applied in this situation:\n\n**Time Series Analysis Techniques**\n\n1. **Moving Averages:**  Calculate the average of stock prices over a specific period (e.g., 5 days, 10 days, 20 days). This smooths out short-term fluctuations and helps identify longer-term trends.\n\n2. **Seasonal Trends:** Look for patterns in stock prices that repeat over specific time intervals (e.g., daily, weekly, monthly). This helps understand seasonality in the stock\'s behavior.\n\n3. **Volatility Patterns:*

In [6]:
from flask import Flask, jsonify, request
import requests

app = Flask(__name__)

# Predefined variables
data = {
    'userId': 'cqtf47@motorolasolutions.com',
    'model': 'VertexGemini',
    'prompt': "Given the historical stock price data of Adidas from the datastore, analyze the trends in daily highs and lows over the past months. Use time series analysis techniques such as moving averages, seasonal trends, and volatility patterns to forecast the stock's high and low values for 14th October 2024. Don't consider any external factors like market events or economic conditions. Based on this analysis, predict the approximately expected high and low prices for Adidas stock on 14th October 2024.",
    'datastoreId': '362a5d9c-65f8-43df-a751-2057708501ca'
}

headers = {
    'x-msi-genai-api-key': 'm,(w(7-CzS-kbVY8u3AF.JYeAG-7PLyuyP6.x@zj',
    'Content-Type': 'application/json'
}

url = 'https://genai-service.stage.commandcentral.com/app-gateway/api/v2/chat'

@app.route('/generate-response', methods=['POST'])
def generate_response():
    response = requests.post(url, headers=headers, json=data)
    return jsonify(response.json())

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

c:\Users\CQTF47\AppData\Local\miniconda3\envs\unstructured_env\lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
